In [35]:
import pandas as pd
import numpy as np
from transformers import AutoTokenizer

In [36]:
tokenizer = AutoTokenizer.from_pretrained("D:\\code\\pretrain_model_dir\\bert-base-chinese")

In [53]:
instance = {
    "sentence": ["张", "雁", "冰", "，", "男", "，"],
    "ner": [{"index": [0, 1, 2], "type": "NAME"}],
    "word": [[0, 1, 2], [3], [4], [5]],
}

In [38]:
dis2idx = np.zeros((1000), dtype="int64")
dis2idx[1] = 1
dis2idx[2:] = 2
dis2idx[4:] = 3
dis2idx[8:] = 4
dis2idx[16:] = 5
dis2idx[32:] = 6
dis2idx[64:] = 7
dis2idx[128:] = 8
dis2idx[256:] = 9

In [39]:
length = 6

In [47]:
_dist_inputs = np.zeros((length, length), dtype=int)

In [48]:
for k in range(length):
    _dist_inputs[k, :] += k
    _dist_inputs[:, k] -= k

In [49]:
_dist_inputs

array([[ 0, -1, -2, -3, -4, -5],
       [ 1,  0, -1, -2, -3, -4],
       [ 2,  1,  0, -1, -2, -3],
       [ 3,  2,  1,  0, -1, -2],
       [ 4,  3,  2,  1,  0, -1],
       [ 5,  4,  3,  2,  1,  0]])

In [50]:
for i in range(length):
    for j in range(length):
        if _dist_inputs[i, j] < 0:
            _dist_inputs[i, j] = dis2idx[-_dist_inputs[i, j]] + 9
        else:
            _dist_inputs[i, j] = dis2idx[_dist_inputs[i, j]]
_dist_inputs[_dist_inputs == 0] = 19

In [51]:
_dist_inputs

array([[19, 10, 11, 11, 12, 12],
       [ 1, 19, 10, 11, 11, 12],
       [ 2,  1, 19, 10, 11, 11],
       [ 2,  2,  1, 19, 10, 11],
       [ 3,  2,  2,  1, 19, 10],
       [ 3,  3,  2,  2,  1, 19]])

In [54]:
tokens = [tokenizer.tokenize(word) for word in instance["sentence"]]
pieces = [piece for pieces in tokens for piece in pieces]
_bert_inputs = tokenizer.convert_tokens_to_ids(pieces)
_bert_inputs = np.array([tokenizer.cls_token_id] + _bert_inputs + [tokenizer.sep_token_id])

In [56]:
# 输入的序列长度
length = len(instance["sentence"])
# 不知道这几个矩阵有啥用, 看图应该是核心结构
_grid_labels = np.zeros((length, length), dtype=np.int32)
_pieces2word = np.zeros((length, len(_bert_inputs)), dtype=np.bool_)
_dist_inputs = np.zeros((length, length), dtype=np.int32)
_grid_mask2d = np.ones((length, length), dtype=np.bool_)

In [57]:
if tokenizer is not None:
    start = 0
    for i, pieces in enumerate(tokens):
        if len(pieces) == 0:
            continue
        # [start: start+N], N 是词块的长度
        pieces = list(range(start, start + len(pieces)))
        # 填充 _pieces2word, i 是序列的第 N 个词, [start+1: start+N+2] = 1
        _pieces2word[i, pieces[0] + 1 : pieces[-1] + 2] = 1
        start += len(pieces)

In [58]:
for k in range(length):
    _dist_inputs[k, :] += k
    _dist_inputs[:, k] -= k

In [59]:
_dist_inputs

array([[ 0, -1, -2, -3, -4, -5],
       [ 1,  0, -1, -2, -3, -4],
       [ 2,  1,  0, -1, -2, -3],
       [ 3,  2,  1,  0, -1, -2],
       [ 4,  3,  2,  1,  0, -1],
       [ 5,  4,  3,  2,  1,  0]])

In [60]:
for i in range(length):
    for j in range(length):
        if _dist_inputs[i, j] < 0:
            _dist_inputs[i, j] = dis2idx[-_dist_inputs[i, j]] + 9
        else:
            _dist_inputs[i, j] = dis2idx[_dist_inputs[i, j]]
_dist_inputs[_dist_inputs == 0] = 19

In [61]:
_dist_inputs

array([[19, 10, 11, 11, 12, 12],
       [ 1, 19, 10, 11, 11, 12],
       [ 2,  1, 19, 10, 11, 11],
       [ 2,  2,  1, 19, 10, 11],
       [ 3,  2,  2,  1, 19, 10],
       [ 3,  3,  2,  2,  1, 19]])

In [69]:
for entity in instance["ner"]:
    index = entity["index"]
    print(index)
    for i in range(len(index)):
        print(i, len(index))
        if i + 1 >= len(index):
            print("break", i)
            break
        _grid_labels[index[i], index[i + 1]] = 1
    # _grid_labels[index[-1], index[0]] = vocab.label_to_id(entity["type"])
    _grid_labels[index[-1], index[0]] = 10

[0, 1, 2]
0 3
1 3
2 3
break 2


In [66]:
_grid_labels

array([[ 0,  1,  0,  0,  0,  0],
       [ 0,  0,  1,  0,  0,  0],
       [10,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0]])

In [70]:
def convert_index_to_text(index, type):
    text = "-".join([str(i) for i in index])
    text = text + "-#-{}".format(type)
    return text

In [72]:
convert_index_to_text([0, 1, 2], "NAME")

'0-1-2-#-NAME'